# Fitting growth models to intracellular Mtb load

In [2]:
from pathlib import Path

import btrack
import zarr

from macrohet.growth_model import (
    collate_tracks_to_df,
    compute_doubling_metrics,
    fit_lowess,
    process_mtb_area,
)

# Conversion constant: pixel area in µm²
image_resolution = 1.4949402023919043e-7  # meters per pixel
microns_per_pixel = image_resolution * 1e6  # µm per pixel
pixel_to_mum_sq_scale_factor = microns_per_pixel ** 2  # µm² per pixel

### 1. Load tracks

In [3]:
tracks_output_fn = '../data/tracks.h5'

with btrack.io.HDF5FileHandler(
    tracks_output_fn, 
    'r', 
    obj_type='obj_type_1'
) as reader:
    tracks = reader.tracks


[INFO][2025/06/23 12:31:08 PM] Opening HDF file: ../data/tracks.h5...
[INFO][2025/06/23 12:31:08 PM] Loading tracks/obj_type_1
[INFO][2025/06/23 12:31:08 PM] Loading LBEP/obj_type_1
[INFO][2025/06/23 12:31:08 PM] Loading objects/obj_type_1 (41424, 5) (41424 filtered: None)
[INFO][2025/06/23 12:31:09 PM] Closing HDF file: ../data/tracks.h5


In [6]:
tracks[0]

,ID,t,x,y,z,parent,root,state,generation,dummy,minor_axis_length,mean_intensity,Infected,orientation,area,major_axis_length
0,423,0,723.909912,33.010132,0.000000,423,423,5,0,False,87.624420,"(3,) array",1.0,-0.105442,23203.0,344.046997
1,423,1,727.771362,38.211437,0.000000,423,423,5,0,False,95.545296,"(3,) array",1.0,-0.105384,25659.0,410.356964
2,423,2,730.887634,26.283424,0.000000,423,423,5,0,False,89.392372,"(3,) array",1.0,-0.172310,19483.0,281.636444
3,423,3,732.147583,27.895367,0.000000,423,423,5,0,False,92.556488,"(3,) array",1.0,-0.172292,22840.0,325.991333
4,423,4,730.891357,33.019173,0.000000,423,423,5,0,False,80.582886,"(3,) array",1.0,-0.152742,23224.0,373.707001
5,423,5,731.606384,30.630995,0.000000,423,423,5,0,False,78.869858,"(3,) array",1.0,-0.151112,21159.0,351.986481
6,423,6,732.788086,30.744682,0.000000,423,423,5,0,False,79.728958,"(3,) array",1.0,-0.169930,21750.0,358.099487
7,423,7,733.528625,27.149408,0.000000,423,423,5,0,False,67.987579,"(3,) array",0.0,-0.195798,15457.0,294.958893
8,423,8,733.524414,32.480957,0.000000,423,423,5,0,False,73.575928,"(3,) array",1.0,-0.201907,21431.0,385.717529
9,423,9,735.238953,29.900726,0.000000,423,423,5,0,False,79.070381,"(3,) array",1.0,-0.251289,22385.0,409.529633


### 2. Convert tracks to DataFrame

In [ ]:
df = collate_tracks_to_df(tracks, expt_ID="EXP1", acq_ID=(1, 1), pixel_to_mum_sq_scale_factor=pixel_to_mum_sq_scale_factor)

In [ ]:
df

### Clean and smooth intracellular Mtb area

In [ ]:
df = process_mtb_area(df, window=10, spike_threshold=2.0)

### Fit LOWESS growth models to Mtb growth

In [ ]:
df = fit_lowess(df, frac=0.25)

### Compute doubling metrics from growth model

In [ ]:
df = compute_doubling_metrics(df)

### Export tracks with single-cell quantifications

#### Use CSV

In [ ]:
# Save as CSV
df.to_csv("../data/single_cell_df.csv", index=False)

#### Use Zarr

In [ ]:
# Save as Zarr (column-wise store)
df_zarr_path = Path("../data/single_cell_df.zarr")
zarr.save_group(str(df_zarr_path), df.to_dict(orient="list"), overwrite=True)